In [1]:
import pandas as pd
from gurobipy import *
import openpyxl

In [2]:
# reading the inputs from Excel workbook
wb = openpyxl.load_workbook("Data.xlsx")
main = wb['Main']

In [3]:
# read objevtive
obj = main.cell(column=2, row=1).value
if obj =='max':
    obj = -1
elif obj =='min':
    obj = 1
else:
    raise NameError('Objective cannot be recognized')
# read obj coefficient
obj_coeff = []
for c in range(2,10000):
    v = main.cell(column=c, row=3).value
    if v is None:
        break
    else:
        obj_coeff.append(v)
n_variables = len(obj_coeff)

#read variable properties
variable_name = []
for c in range(n_variables):
    v = main.cell(column=c+2, row=2).value
    if v is None:
        variable_name.append('x'+str(c+1))
    elif type(v) != str:
        variable_name.append(str(v))
    else:
        variable_name.append(v)
        
variable_type = []
for c in range(n_variables):
    v = main.cell(column=c+2, row=4).value
    if v is None:
        t = 'C' 
    elif v == 'cont':
        t = 'C'
    elif v == 'int':
        t = 'I'
    elif v == 'bin':
        t = 'B'
    else:
        raise NameError('Variable type cannot be recognized')
    variable_type.append(t)
    
variable_lb = []
for c in range(n_variables):
    v = main.cell(column=c+2, row=5).value
    if v is None:
        t = 0
    elif v == '"+inf"':
        raise NameError('Variable lower bound cannot be +inf')
    elif v == '"-inf"':
        t = -GRB.INFINITY
    elif type(v) in [float, int]:
        t = v
    else:
        raise NameError('Variable lower bound cannot be recognized')
    variable_lb.append(t)

variable_ub = []
for c in range(n_variables):
    v = main.cell(column=c+2, row=6).value
    if v is None:
        t = GRB.INFINITY
    elif v == '"+inf"':
        t = GRB.INFINITY
    elif v == '"-inf"':
        raise NameError('Variable upper bound cannot be -inf')
    elif type(v) in [float, int]:
        t = v
    else:
        raise NameError('Variable upper bound cannot be recognized')
    variable_ub.append(t)
#print(obj_coeff)

In [4]:
# read constraints
## find constraint column index
for c in range(1, n_variables+100):
    v = main.cell(column=c, row=8).value
    if v == 'constraint type':
        const_typ_cind = c
        break

if const_typ_cind-2 != n_variables:
    raise NameError('Number of variables does not match the constraints')
    
## find last constraint row index
for r in range(9, 100000):
    v = main.cell(column=const_typ_cind, row=r).value
    if v is None:
        break
    elif v not in ["<=", ">=", "="]:
        raise NameError("Wrong constraint type")
    else:
        last_r_consraints=r

## read constraints as dataframe
for r in range(9, last_r_consraints+1):
    v = main.cell(column=const_typ_cind+2, row=r).value
    if v is None:
        main.cell(column=const_typ_cind+2, row=r).value = "c"+str(r-8)
    elif type(v) != str:
        main.cell(column=const_typ_cind+2, row=r).value = str(v)
    else:
        1==1        
values = []
cons_df = pd.DataFrame()
for r in range(9, last_r_consraints+1):
    r_values = []
    for c in range(2, const_typ_cind+3):
        r_values.append(main.cell(column=c, row=r).value)
    values.append(r_values)
    cons_df = pd.DataFrame(values)
    cons_df.fillna(0, inplace=True)
    cons_df.rename(columns={const_typ_cind-2:'constraint type', 
                            const_typ_cind-1:'RHS values', 
                            const_typ_cind:'constraint name'}, inplace=True)

In [5]:
# setup model
m = Model()

Restricted license - for non-production use only - expires 2023-10-25


In [6]:
# add variables
x=m.addVars(n_variables)
# set types, lb, ub of variables
for i in range(n_variables):
    x[i].setAttr('VarNAME', variable_name[i])
    x[i].setAttr('vType', variable_type[i])
    x[i].setAttr('lb', variable_lb[i])
    x[i].setAttr('ub', variable_ub[i])

In [7]:
# set objective
objective = quicksum(obj_coeff[i] * x[i] for i in range(n_variables))
m.setObjective(objective, obj)

In [8]:
# add constraints
if cons_df.shape[0]:
    ## add le constraints
    le_const = cons_df[cons_df['constraint type'] == '<=']
    le_const.reset_index(inplace=True, drop=True)
    for i in range(le_const.shape[0]):
        m.addConstr(quicksum(le_const.iloc[i,j]* x[j] for j in range(n_variables)) <= le_const['RHS values'][i], 
                    name = le_const['constraint name'][i])
    ## add ge constraints
    ge_const = cons_df[cons_df['constraint type'] == '>=']
    ge_const.reset_index(inplace=True, drop=True)
    for i in range(ge_const.shape[0]):
        m.addConstr(quicksum(ge_const.iloc[i,j]* x[j] for j in range(n_variables)) >= ge_const['RHS values'][i], 
                    name = ge_const['constraint name'][i])
    ## add eq constraints
    eq_const = cons_df[cons_df['constraint type'] == '=']
    eq_const.reset_index(inplace=True, drop=True)
    for i in range(eq_const.shape[0]):
        m.addConstr(quicksum(eq_const.iloc[i,j]* x[j] for j in range(n_variables)) == eq_const['RHS values'][i], 
                    name = eq_const['constraint name'][i])

In [9]:
## run the model
m.optimize()

Gurobi Optimizer version 9.5.2 build v9.5.2rc0 (mac64[x86])
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads
Optimize a model with 71 rows, 50 columns and 170 nonzeros
Model fingerprint: 0xab53bc53
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [3e+00, 2e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 6e+00]
Presolve removed 57 rows and 23 columns
Presolve time: 0.08s
Presolved: 14 rows, 27 columns, 60 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    1.8400000e+02   2.200000e+01   0.000000e+00      0s
Extra simplex iterations after uncrush: 2
      12    2.5000000e+02   0.000000e+00   0.000000e+00      0s

Solved in 12 iterations and 0.18 seconds (0.00 work units)
Optimal objective  2.500000000e+02


In [10]:
# print optimal solution
m.printAttr('X')


    Variable            X 
-------------------------
        HHC1            1 
        HSC1            1 
        LHC1            1 
        LM21            1 
        LP11            1 
        LSC1            1 
         HM2            1 
        HSC2            1 
        LHC2            1 
        LM12            1 
        LM22            1 
        LSC2            1 
        HHC3            1 
        HSC3            1 
        LHC3            1 
        LM13            1 
        LM23            1 
        LSC3            1 
         HP4            1 
        HSC4            1 
        LM14            1 
        LM24            1 
        LP14            1 
        LSC4            1 
        HHC5            1 
        HSC5            1 
        LHC5            1 
        LM15            1 
        LM25            1 
        LSC5            1 


In [11]:
# sensitivity analysis
for v in m.getVars():
    print("For Variable " + v.VarName+ ":")
    print("Minimum value coefficient can take before the optimal decision changes "  + "is " + str(v.SAObjLow))
    print("Maximum value coefficient can take before the optimal decision changes "  + "is " + str(v.SAObjUp))

For Variable HHC1:
Minimum value coefficient can take before the optimal decision changes is 10.0
Maximum value coefficient can take before the optimal decision changes is inf
For Variable HM1:
Minimum value coefficient can take before the optimal decision changes is 7.0
Maximum value coefficient can take before the optimal decision changes is inf
For Variable HP1:
Minimum value coefficient can take before the optimal decision changes is 12.0
Maximum value coefficient can take before the optimal decision changes is inf
For Variable HSC1:
Minimum value coefficient can take before the optimal decision changes is 10.0
Maximum value coefficient can take before the optimal decision changes is 15.0
For Variable LHC1:
Minimum value coefficient can take before the optimal decision changes is -inf
Maximum value coefficient can take before the optimal decision changes is 13.0
For Variable LM11:
Minimum value coefficient can take before the optimal decision changes is 3.0
Maximum value coefficien

Maximum value coefficient can take before the optimal decision changes is inf
For Variable LP25:
Minimum value coefficient can take before the optimal decision changes is 8.0
Maximum value coefficient can take before the optimal decision changes is inf
For Variable LSC5:
Minimum value coefficient can take before the optimal decision changes is -inf
Maximum value coefficient can take before the optimal decision changes is 8.0


In [12]:
for c in m.getConstrs():
    print("For constraint " + c.ConstrName+ ":")
    print("Shawdow Price is " + str(c.pi))
    print("Minimum value RHS can take before the shadow price changes "  + "is " + str(c.SARHSLow))
    print("Maximum value RHS can take before the shadow price changes "  + "is " + str(c.SARHSUp))

For constraint No HHC duplicate:
Shawdow Price is 0.0
Minimum value RHS can take before the shadow price changes is 1.0
Maximum value RHS can take before the shadow price changes is inf
For constraint No HM duplicate:
Shawdow Price is 0.0
Minimum value RHS can take before the shadow price changes is 0.0
Maximum value RHS can take before the shadow price changes is inf
For constraint No HP duplicate:
Shawdow Price is 0.0
Minimum value RHS can take before the shadow price changes is 0.0
Maximum value RHS can take before the shadow price changes is inf
For constraint No HSC duplicate:
Shawdow Price is 0.0
Minimum value RHS can take before the shadow price changes is 1.0
Maximum value RHS can take before the shadow price changes is inf
For constraint No LHC duplicate:
Shawdow Price is -5.0
Minimum value RHS can take before the shadow price changes is 1.0
Maximum value RHS can take before the shadow price changes is 1.0
For constraint No LM1duplicate:
Shawdow Price is 0.0
Minimum value RHS 